### Group By, Having & Count

Group By always comes after From w or w/o where

Count, Sum, Max, etc are basic aggregate operators

Having is a common type of clause


In [1]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
import os

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/xlyue/Documents/kaggle learning/Intro to SQL/sql-bq-244804-803e47cbe74e.json"

client = bigquery.Client()
dataset_ref = client.dataset('hacker_news', project = 'bigquery-public-data')
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [3]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)
print()
print(len(tables))

comments
full
full_201510
stories

4


In [4]:
table_ref = dataset_ref.table('comments')
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


Find which comment generates most replies

In [4]:
query_popular = """
                       SELECT parent, COUNT(id)
                       FROM `bigquery-public-data.hacker_news.comments`
                       GROUP BY parent
                       HAVING COUNT(id) > 10
                       """

In [5]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
query_job = client.query(query_popular, job_config = safe_config)

In [6]:
# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

In [7]:
# Print the first five rows of the DataFrame
popular_comments.head()

,parent,f0_
0,3873271,50
1,5308611,63
2,5176140,63
3,9269660,51
4,9439286,69


In [8]:
query_good = """
                   SELECT parent, COUNT(id)
                   FROM `bigquery-public-data.hacker_news.comments`
                   GROUP BY parent
                   """

Exercises

In [11]:
prolific_commenters_query = """
                                         SELECT author, COUNT(id) AS NumPosts
                                         FROM `bigquery-public-data.hacker_news.comments`
                                         GROUP BY author
                                         HAVING COUNT(id) > 10000
                                         """

In [12]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**9)
query_job = client.query(prolific_commenters_query, job_config = safe_config)

# API request - run the query, and return a pandas DataFrame
prolific_commenters = query_job.to_dataframe()

# View top few rows of results
print(prolific_commenters.head())

    author  NumPosts
0      eru     10448
1  rbanffy     10557
2    DanBC     12902
3    sp332     10882
4   davidw     10764


How many comments have been deleted? 

(If a comment was deleted, the `deleted` column in the comments table will have the value `True`.)

In [13]:
query = """
           select count(deleted)
           from `bigquery-public-data.hacker_news.comments`
           where deleted = True
           """
query_job = client.query(query)

# API request - run the query, and return a pandas DataFrame
deleted_posts = query_job.to_dataframe()

# View results
print(deleted_posts)

      f0_
0  227736


#### Note
Sometimes we are unsure about what to put inside the COUNT() function, 

what we can do is using COUNT(1) to count the rows in each group.